# M1 BIF - TP2
# Tableau des suffixes

## Utilitaire : générateur de séquences

/!\ Prévoir tests et evaluation des temps de calculs sur des séquences différentes

In [1]:
import time
import random

from fasta import get_seq_from_fasta
from print_table import print_table
from kasai import lcp_kasai

In [2]:
#Methode pour obtenir les performances d'une fonction

#usage
#seq = get_seq_from_fasta("file.fasta",10000)
#print_table(s,sa,lcp)

def timer(f, **kwargs):
    t0 = time.perf_counter()
    ret = f(**kwargs)
    t1 = time.perf_counter()
    print(f'{f.__name__} took {round(t1-t0,6)} seconds.')

def test(a,b):
    return a+b

timer(test, a=2, b=3)

test took 2e-06 seconds.


#### Q1. Ecrire un générateur de séquences aléatoires sur l'alphabet _{a,t,c,g}_. La fonction prendra en paramètre la taille e la séquence à générer. On utilisera des fonctions de la librairie _random_

In [3]:
"""
Generer une sequence aleatoire de taille n
size : taille de la sequence
eos : caractère 'end of string', par défaut ''
"""

def generateSequence(size:int, eos='') -> str:
    seq =""
    for i in range(size):
        seq += random.choice('ACGT')
    seq += eos
    return seq

def get_seq_bis(N:int, eos='') -> str:
    return ''.join([random.choice('ACGT') for _ in range(N)]) + eos
    

timer(generateSequence, size = 10)

generateSequence took 3.6e-05 seconds.


In [4]:
seq01 = generateSequence(10)
print(seq01)
seq02 = generateSequence(10,'$')
print(seq02)

AAACCGCCGG
GCCGCCAGGT$


## 1. Tableau des suffixes, LCP

#### Q2. Implémentez l'ensemble des fonctions nécessaires au calcul du tableau des suffixes pour une séquence S.

In [5]:
"""
getSuffixes
return all suffixes of a string, in a list
text -> the string where we get the suffixes
"""
def getSuffixes(text):
    suffixes = {}
    size = len(text)
    for i in range(size):
        suffixes[i] = text[i:]
    return suffixes

timer(getSuffixes,text=seq02)

getSuffixes took 1.3e-05 seconds.


In [6]:
suffixes01 =  getSuffixes(seq02)
suffixes01

{0: 'GCCGCCAGGT$',
 1: 'CCGCCAGGT$',
 2: 'CGCCAGGT$',
 3: 'GCCAGGT$',
 4: 'CCAGGT$',
 5: 'CAGGT$',
 6: 'AGGT$',
 7: 'GGT$',
 8: 'GT$',
 9: 'T$',
 10: '$'}

In [7]:
"""
getSortedSuffixes
return the list of suffixes ordered alphabetically ($<A<C<G<T)
suffixes -> list of the suffixes to order
"""

def getSortedSuffixes(suffixes, reverse = False):
    return sorted(suffixes.items(), key=lambda x : x[1], reverse=reverse)

In [8]:
sortedSuffixes01 = getSortedSuffixes(suffixes01)
print(sortedSuffixes01)


[(10, '$'), (6, 'AGGT$'), (5, 'CAGGT$'), (4, 'CCAGGT$'), (1, 'CCGCCAGGT$'), (2, 'CGCCAGGT$'), (3, 'GCCAGGT$'), (0, 'GCCGCCAGGT$'), (7, 'GGT$'), (8, 'GT$'), (9, 'T$')]


In [9]:
sortedSuffixesReversed01 = getSortedSuffixes(suffixes01,True)
print(sortedSuffixesReversed01)

[(9, 'T$'), (8, 'GT$'), (7, 'GGT$'), (0, 'GCCGCCAGGT$'), (3, 'GCCAGGT$'), (2, 'CGCCAGGT$'), (1, 'CCGCCAGGT$'), (4, 'CCAGGT$'), (5, 'CAGGT$'), (6, 'AGGT$'), (10, '$')]


In [10]:
"""
getSuffixesTable
return only the indexes of the previously sorted suffix dict
"""
def getSuffixesTable(sortedSuffixes):
    table = []
    for elem in sortedSuffixes:
        table.append(elem[0])
    return table

In [12]:
suffixTable01 = getSuffixesTable(sortedSuffixes01)
print(suffixTable01)
suffixTable02 = getSuffixesTable(sortedSuffixesReversed01)
print(suffixTable02)

[10, 6, 5, 4, 1, 2, 3, 0, 7, 8, 9]
[9, 8, 7, 0, 3, 2, 1, 4, 5, 6, 10]


In [13]:
# Full fonction
#Get all suffixes -> Sort them -> return SuffixArray

def getSA(text, rev = False):
    suffixes = getSuffixes(text)
    sortedSuffixes = getSortedSuffixes(suffixes,rev)
    suffixTable = getSuffixesTable(sortedSuffixes)
    return suffixTable
    

In [14]:
print(seq02)

sa01 = getSA(seq02)
print(sa01)
sa02 = getSA(seq02, True)
print(sa02)

GCCGCCAGGT$
[10, 6, 5, 4, 1, 2, 3, 0, 7, 8, 9]
[9, 8, 7, 0, 3, 2, 1, 4, 5, 6, 10]


#### Q3. Implémentez une fonction de calcul de LCP : coder la fonction _get_lcp(s,sa)_ renvoyant le tableau _lcp_ tel que _lcp[i]_ est ele plus long préfixe commnu de _s[sa[i - 1],n])_.

In [15]:
"""
Return the array of  longest common prefix between two consecutives suffixes
text -> the text that contains the suffixes
suffixArray -> the indexes of the suffixes, sorted alphabetically
"""

def get_lcp(text, suffixArray):
    length = len(suffixArray)
    lcp = [0,0]
    
    #Compare i with the previous suffix
    for i in range(2,length):
        j = 0
        #Test both suffixes letter by letter
        while text[suffixArray[i - 1]+j] == text[suffixArray[i]+j]:
            j +=1
        lcp.append(j)
    
    return lcp
        
        
        

In [16]:
lcp01 = get_lcp(seq02,sa02)
lcp01

[0, 0, 1, 1, 3, 0, 1, 2, 1, 0, 0]

#### Q4.  Essayez vos implémentations de SA et LCP sur des séquences aléatoires de taille 1000,10 000, 100 000 (1 000 000 ?).

In [17]:
#n -> le nombre de lettres a generer dans la sequence
def testSaLcp(n, print= False):
    seq03 = generateSequence(n,'$')
    sa03 = getSA(seq03)
    lcp03 = get_lcp(seq03,sa03)
    if (print):
        print(f'Sequence générée : {seq03}\n')
        print(f'Liste des suffixes trié : {sa03}\n')
        print(f'Liste des lcp : {lcp03}\n')


In [18]:
testSaLcp(10)

In [19]:
#Test pour 1000
timer(testSaLcp,n=1000)

testSaLcp took 0.0043 seconds.


In [20]:
#Test pour 10 000
timer(testSaLcp,n=10000)

testSaLcp took 0.11634 seconds.


In [21]:
#Test pour 100 000
#timer(testSaLcp,n=100000)

#TOO LONG

## 2. Le plus grand mot répété

#### Q6. En utilisant le tableau des suffixes et son _lcp_ , implémenter une fonction _longest_repeat(s, sa, lcp)_ qui renvoie les plus grands mots répétés de _S_ ainsi que leurs positions dans _S_.

In [22]:
#Obtenir le plus long mot
def getLongestRepeat(S,SA,LCP):
    size = len(SA)
    repeatMax= 0
    stringMax = ""
    for i in range(size):
        if LCP[i] > repeatMax:
            repeatMax = LCP[i];
            stringMax = S[SA[i]:SA[i]+repeatMax]
    return stringMax

In [23]:
size = 10
seq04 = generateSequence(size,'$')
sa04 = getSA(seq04)
lcp04 = get_lcp(seq04,sa04)
    
lr01 = getLongestRepeat(seq04,sa04,lcp04)
lr01

'A'

In [24]:
#Obtenir leS plus longS motS

In [25]:
#Obtenir le plus long mot
def getListLongestRepeat(S,SA,LCP):
    size = len(SA)
    repeatMax= 0
    stringMax = dict()
    for i in range(size):
        if LCP[i] == repeatMax:
            stringMax[S[SA[i]:SA[i]+repeatMax]] = [SA[i-1], SA[i]] 
        if LCP[i] > repeatMax:
            repeatMax = LCP[i];
            stringMax.clear()
            stringMax[S[SA[i]:SA[i]+repeatMax]] = [SA[i-1], SA[i]] 
    return stringMax

In [26]:
size = 10
seq04 = generateSequence(size,'$')
sa04 = getSA(seq04)
lcp04 = get_lcp(seq04,sa04)
    
lr01 = getListLongestRepeat(seq04,sa04,lcp04)
lr01

{'A': [4, 5], 'C': [8, 2], 'G': [7, 6], 'T': [1, 0]}

#### Q7. Comparer le résultat de cette fonction pour des séquences biologiques et pour des séquences aléatoires. La séquence biologique se trouve dans le fichier MC48.fasta (séquence du génome de la bactérie Neisseria meningitidis strain MC58) (utiliser la fonction get_seq_from_fasta dans le fichier fasta.py)

In [27]:
"""
Obtenir une sequence de taille n depuis le fichier fasta
size : taille de la sequence
"""

def getSequence(size):
    file_path = "./MC58.fasta"
    seq = get_seq_from_fasta(file_path,size) + '$'
    return seq

In [28]:
fastaSeq01 = getSequence(100000)
print(fastaSeq01[:100],"...")

fastaSeqSA01 = getSA(fastaSeq01)
fastaSeqLCP01 = get_lcp(fastaSeq01,fastaSeqSA01)
    
fastaSeqLR01 = getListLongestRepeat(fastaSeq01,fastaSeqSA01,fastaSeqLCP01)
print(fastaSeqLR01)

TTCGGCTTAAACCTTATCCACATCCAAACGCATAACCGTAACCCATTCACCGTTATGGAAATGTCGCCCGACAACCACCCAGCCGAATGATTCATAAAAT ...
{'CGTCATTCCCGCGCAGGCGGGAATCTAGAACGTAAAATCTAAAGAAACCGTGTTGTAACGGCAGACCGATGCCGTCATTCCCGCGCAGGCGGGAATCTAGACCATTGGACAGCGGCAATATTCAAAGATTATCTGAAAGTCCGAGATTCTGGATTCCCACTTTCGTGGGAATGACGGGATTTGAGATTGCGGCATTTATCGGAAAAAACAGAAACCGCTCCGCCGTCATTCCCGCGCAGGCGGGAATC': [14341, 44640]}


## 3. Pattern matching

#### Q8. Coder la fonction _search(p,s,sa)_ renvoyant :
  - la position d'**une** occurence du mot *p* dans *s* si elle existe
  - -1 sinon

In [30]:
"""
Compare two strings
return the difference as integer
"""
def cmp(pattern,string,pLength):
    for i in range(pLength):
        if pattern[i] != string[i]:
            return ord(pattern[i]) - ord(string[i])
    return 0

In [31]:
def searchPatternOccurence(pattern, string, suffixArray):
    sLength = len(string)
    pLength = len(pattern)
    leftBound = 0
    rightBound = sLength -1
    
    while leftBound <= rightBound:
        mid = leftBound + (rightBound - leftBound)//2
        match = cmp(pattern,string[suffixArray[mid]:],pLength) # pass substring ?
        if match == 0:
            return suffixArray[mid]
        elif match < 0:
            rightBound = mid - 1
        else:
            leftBound = mid +1
    
    return -1

In [32]:
pattern = 'CGTCATTCCCGCGCAGGCGGGAATCTAGAACGTAAAATCTAAAGAAACCGTGTTGTAACGGCAGACCGATGCCGTCATTCCCGCGCAGGCGGGAATCTAGACCATTGGACAGCGGCAATATTCAAAGATTATCTGAAAGTCCGAGATTCTGGATTCCCACTTTCGTGGGAATGACGGGATTTGAGATTGCGGCATTTATCGGAAAAAACAGAAACCGCTCCGCCGTCATTCCCGCGCAGGCGGGAATC'
answer = searchPatternOccurence(pattern,fastaSeq01,fastaSeqSA01)
print(answer)

44640


#### Q9. Coder la fonction *search_pos(p, s, sa)* renvoyant :
  - les positions **des** occurences du mot *p* dans *s* si elles existent
  - -1 sinon

In [33]:
def searchPatternMultipleOccurences(pattern, string, suffixArray):
    sLength = len(string)
    pLength = len(pattern)
    leftBound = 0
    rightBound = sLength -1
    
    #Find left bound
    while leftBound < rightBound:
        mid = leftBound + (rightBound - leftBound)//2
        match = cmp(pattern,string[suffixArray[mid]:],pLength) # pass substring ?
        if match > 0:
            leftBound = mid +1
        else:
            rightBound = mid

    leftBoundFound = leftBound
    rightBound = pLength -1
    
    #find right bound
    while leftBound < rightBound:
        mid = leftBound + (rightBound - leftBound)//2
        match = cmp(pattern,string[suffixArray[mid]:],pLength) # pass substring ?
        if match == 0:
            leftBound = mid + 1
        else:
            rightBound = mid
    
    #return
    return (leftBoundFound, rightBound - 1)
    

In [34]:
answer = searchPatternMultipleOccurences(pattern,fastaSeq01,fastaSeqSA01)
print(answer)

(42733, 246)


#### Q10. Comparer les performances de la fonction avec votre implémentation de Pattern Matching sans indexation (version naïve et Morris Pratt)

In [36]:
timer(searchPatternMultipleOccurences,pattern=pattern,string=fastaSeq01,suffixArray=fastaSeqSA01)

searchPatternMultipleOccurences took 0.000561 seconds.


### Imports from TP1

In [38]:
# creer un hash d'une string
# fait la somme des valeurs ascii de chaque lettre
def hash_value(s,p):
    s = s.upper()
    hash = 0
    for i in range(len(s)):
        hash += ord(s[i]) * p**i
    return hash

# pass substring of text as parameter
# len(string) = len(pattern)+1, to get previous and next letter
def update_hash_value2(previous,S,prime,pLen):
    S = S.upper()
    hash = previous - ord(S[0])
    hash //= prime
    hash += ord(S[pLen]) * prime**(pLen-1)
    return hash

#RabinKarp, passing substring of text as parameter in update_hash function
def RabinKarp2(P,T,p):
    pattern = P.upper()
    text = T.upper()
    pLen = len(pattern)
    tLen = len(text)
    assert pLen <= tLen
    matchList = []
    
    hash_pattern = hash_value(pattern,p)
    hash_text = hash_value(text[:pLen],p)
    
    for i in range (tLen - pLen+1):
        if(hash_pattern == hash_text):
            j = 0
            while (j<pLen):
                if (text[i+j] != pattern[j]):
                    break
                j+=1
            if (j==pLen):
                matchList.append(i)
                
        if(i < tLen - pLen):
            hash_text = update_hash_value2(hash_text,text[i:i+pLen+1],p,pLen) #Changed the update_hash function
    return matchList

In [40]:
timer(RabinKarp2,P=pattern,T=fastaSeq01,p=11)

RabinKarp2 took 0.354219 seconds.


In [41]:
def naive_pattern_matching(pattern, text):
    pattern = pattern.upper() #pattern a rechercher
    text = text.upper() #texte où rechercher le pattern
    pLen = len(pattern) #longueur du pattern
    tLen = len(text)    #longueur du texte
    match = []
    
    for i in range (tLen - pLen+1):  #parcourir le texte jusqu'a ce que le pattern ne rentre plus dans le texte restant

        j = 0  #comparer les lettres du pattern et du texte une a une
        while (j<pLen):
            if (text[i+j] != pattern[j]):  #si les lettres ne correspondent pas, exit de la boucle while
                break
            j+=1
            
        if (j==pLen):  #si le pattern correspond
            match.append(i)  #ajouter a la liste des occurences
    return match

In [44]:
timer(naive_pattern_matching,pattern=pattern,text=fastaSeq01)

naive_pattern_matching took 0.066705 seconds.


### Conclusion
searchPatternMultipleOccurences took 0.000561 seconds.  
naive_pattern_matching took 0.066705 seconds.  
RabinKarp2 took 0.354219 seconds.  